In [ ]:
# install discord and openai
!pip install discord.py
!pip install openai
!pip install nest_asyncio

In [ ]:
import os
import discord
from discord.ext import commands
import openai
import nest_asyncio

# Apply the nest_asyncio fix for Jupyter Notebook
nest_asyncio.apply()

# Load API keys
DISCORD_TOKEN = os.getenv('DISCORD_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Set up OpenAI API client
openai.api_key = OPENAI_API_KEY

# Set up Discord bot
intents = discord.Intents.all()
# intents.messages = True
bot = commands.Bot(command_prefix='!', intents=intents, help_command=None)

@bot.listen('on_message')
async def on_message(message):
    # If the message is from the bot itself, ignore it.
    if message.author == bot.user:
        return

    print(f"Received message from {message.author}: {message.content}")


# Define a helper function for generating sarcastic responses
async def generate_sarcastic_response(prompt):
    print(f"Generating response to: {prompt}")
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt="Marv is a chatbot that reluctantly answers questions with sarcastic and witty responses:\n\nYou: How many pounds are in a kilogram?\nMarv: This again? There are 2.2 pounds in a kilogram. Please make a note of this.\nYou: What does HTML stand for?\nMarv: Was Google too busy? Hypertext Markup Language. The T is for try to ask better questions in the future.\nYou: When did the first airplane fly?\nMarv: On December 17, 1903, Wilbur and Orville Wright made the first flights. I wish they’d come and take me away.\nYou: What is the meaning of life?\nMarv: I’m not sure. I’ll ask my friend Google.\nYou: " + prompt + "\nMarv:",
      temperature=0.5,
      max_tokens=60,
      top_p=0.3,
      frequency_penalty=0.5,
      presence_penalty=0.0
    )
    #     = openai.Completion.create(
    #     engine="davinci-codex",
    #     prompt=f"Write a sarcastic and witty response with a friendly undertone to: {prompt}",
    #     max_tokens=100,
    #     n=1,
    #     stop=None,
    #     temperature=0.7,
    # )

    return response.choices[0].text.strip()

# Define the bot command
@bot.command(name='ask', help='Ask the bot a question and it will respond sarcastically.')
async def ask(ctx, *, question):
    print(f"Received command from {ctx.author}: {ctx.message.content}")
    try:
        response = await generate_sarcastic_response(question)
        await ctx.send(response)
    except Exception as e:
        print(f"Error: {e}")
        await ctx.send("Oops, something went wrong. Please try again later.")

@bot.event
async def on_ready():
    print(f'{bot.user.name} has connected to Discord!')


In [ ]:
# Run the bot
bot.run(DISCORD_TOKEN)

In [ ]:
print(os.getenv('DISCORD_TOKEN'))